# Import library

In [4]:
# get data
from binance.spot import Spot 

# handle data
import pyspark.sql as ps
from pyspark.sql.functions import from_unixtime,date_format,from_utc_timestamp
from pyspark.sql.types import DateType

# train data
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

# enviroment
import os
from dotenv import load_dotenv
load_dotenv("../../env/app.env")

True

# Setup clients

In [5]:
BINANCE_API_KEY = os.environ.get("BINANCE_API_KEY")
BINANCE_API_SECRET = os.environ.get("BINANCE_API_SECRET")
client = Spot(key=BINANCE_API_KEY, secret=BINANCE_API_SECRET)
# spark = ps.SparkSession.\
#         builder.\
#         appName("pyspark-notebook").\
#         master("spark://spark-master:7077").\
#         config("spark.executor.memory", "4096m").\
#         getOrCreate()
# spark.sparkContext.setLogLevel("WARN")

22/11/30 01:38:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Fetch day period data

### Declare period, symbol and header

In [6]:
DAY_TO_QUERY = 1800
DURATION_EACH_DAY = 60 * 60 * 24 * 1000

In [7]:
PERIOD = "1d"
cols =[
        'Kline open time',   #if a range of time
        'Open price',       #First trade price in the time range
        'High price',       #max buy in time
        'Low price',        #Min sell in time
        'Close price',      #Last trade price in the time range
        'Volume',           #Sum in time
        'Kline Close time',
        'Quote asset volume',   #Sum in time
        'Number of trades',     #Sum in time
        'Taker buy base asset volume',  #Sum in time
        'Taker buy quote asset volume', #Sum in time
        'Unused field, ignore'] # not use 

### Store function

In [8]:
# write dataframe to hdfs with partitions
# def writeToHFDS(df, symbol):
#     df = df \
#         .withColumn("open_time", df["open_time"] / 1000) \
#         .withColumn("date", from_unixtime('open_time').cast(DateType()))
#     df = df \
#         .withColumn("day", date_format(df['date'], 'dd')) \
#         .withColumn("month", date_format(df['date'], 'MM')) \
#         .withColumn("year", date_format(df['date'], 'yyyy'))

#     df.write.option("header",True) \
#          .mode("overwrite") \
#          .partitionBy("day","month","year") \
#          .parquet(f"hdfs://hadoop-namenode:9000/crypto/price/{symbol}/{PERIOD}.parquet")


### Get single symbol data points

In [6]:
today = datetime.utcnow().replace(hour=7, minute=0, second=0, microsecond=0)
today_timestamp = int(today.timestamp() * 1000)

start_timestamp = today_timestamp - DAY_TO_QUERY * DURATION_EACH_DAY
end_timestamp = today_timestamp

for timestamp in range(start_timestamp, end_timestamp, DURATION_EACH_DAY * 100):
    data = client.klines(symbol, PERIOD, limit=100, startTime=timestamp, endTime=timestamp + DURATION_EACH_DAY * 100)
    if start_timestamp == timestamp:
        df = spark.createDataFrame(data, schema=columns)
    else:
        df = df.union(spark.createDataFrame(data))

writeToHFDS(df, symbol)

### Get data points of symbols 

In [9]:
exchanges = client.exchange_info()
symbols = list(map(lambda item: item.get('symbol'), exchanges.get('symbols')))

In [10]:
print(len(symbols))
symbols[:10]

2124


['ETHBTC',
 'LTCBTC',
 'BNBBTC',
 'NEOBTC',
 'QTUMETH',
 'EOSETH',
 'SNTETH',
 'BNTETH',
 'BCCBTC',
 'GASBTC']

> Totally

In [12]:
print(f"{len(symbols)} symbol X {DAY_TO_QUERY} data points = {DAY_TO_QUERY*len(symbols)} data points")

2124 symbol X 1800 data points = 3823200 data points


In [13]:
today = datetime.utcnow().replace(hour=7, minute=0, second=0, microsecond=0)
today_timestamp = int(today.timestamp() * 1000)

start_timestamp = today_timestamp - DAY_TO_QUERY * DURATION_EACH_DAY
end_timestamp = today_timestamp

for symbol in symbols:
    try:
        for timestamp in range(start_timestamp, end_timestamp, DURATION_EACH_DAY * 100):
            data = client.klines(symbol, PERIOD, limit=100, startTime=timestamp, endTime=timestamp + DURATION_EACH_DAY * 100)
            if start_timestamp == timestamp:
                df = spark.createDataFrame(data, schema=columns)
            else:
                df = df.union(spark.createDataFrame(data))

        writeToHFDS(df, symbol)
    except:
        print(f"Error at {symbol}\n")

Error at BCCBTC



Error at HSRBTC

Error at OAXETH

Error at DNTETH

Error at MCOETH

Error at ICNETH

Error at MCOBTC



Error at WTCETH



Error at YOYOBTC



Error at STRATBTC

Error at STRATETH

Error at SNGLSBTC

Error at SNGLSETH

Error at BQXBTC

Error at BQXETH



...

Error at ONTBNB

Error at STORMBTC

Error at STORMETH

Error at STORMBNB

Error at QTUMBNB



Error at XEMETH

Error at XEMBNB



Error at WANBNB

Error at WPRBTC

Error at WPRETH



Error at QLCETH



Error at SYSETH

Error at SYSBNB

Error at QLCBNB

Error at GRSBTC

Error at GRSETH

Error at ADAUSDT

Error at ADABNB

Error at CLOAKBTC

Error at CLOAKETH

Error at GNTBTC

Error at GNTETH

Error at GNTBNB

Error at LOOMBTC

Error at LOOMETH

Error at LOOMBNB

Error at XRPUSDT

Error at BCNBTC

Error at BCNETH

Error at BCNBNB

Error at REPBTC

Error at REPBNB

Error at BTCTUSD

Error at TUSDBTC

Error at ETHTUSD

Error at TUSDETH

Error at TUSDBNB

Error at ZENBTC

Error at ZENETH

Error at ZENBNB

Error at SKYBTC

Error at SKYETH

Error at SKYBNB

Error at EOSUSDT

Error at EOSBNB

Error at CVCBTC

Error at CVCETH

Error at CVCBNB

Error at THETABTC

Error at THETAETH

Error at THETABNB

Error at XRPBNB

Error at TUSDUSDT

Error at IOTAUSDT

Error at XLMUSDT

Error at IOTXBTC

Error at IOTXETH

Error at QKCBTC

Error at QKCETH

Error at AGIBTC

Error at AGIETH

Error at AGIBNB

Error at NXSBTC

Error at NXSETH

Error at NXSBNB

Error at ENJBNB

Error at DATABTC

Error at DATAETH

E

In [14]:
spark.stop()